# Сборный проект: Исследование закономерностей, определяющих успешность компьютерной игры.

Предстоит работа с данными интернет-магазина «Стримчик», который продаёт по всему миру компьютерные игры. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). 

-----------
###### Цель работы: 
Нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

------------
Перед нами данные до 2016 года. Представим, что сейчас декабрь 2016 г., и мы планируем кампанию на 2017-й. Нужно отработать принцип работы с данными.

В наборе данных попадается аббревиатура ESRB (Entertainment Software Rating Board) — это ассоциация, определяющая возрастной рейтинг компьютерных игр. ESRB оценивает игровой контент и присваивает ему подходящую возрастную категорию, например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».

###### Описание данных
- Name — название игры
- Platform — платформа
- Year_of_Release — год выпуска
- Genre — жанр игры
- NA_sales — продажи в Северной Америке (миллионы проданных копий)
- EU_sales — продажи в Европе (миллионы проданных копий)
- JP_sales — продажи в Японии (миллионы проданных копий)
- Other_sales — продажи в других странах (миллионы проданных копий)
- Critic_Score — оценка критиков (максимум 100)
- User_Score — оценка пользователей (максимум 10)
- Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

Данные за 2016 год могут быть неполными.


## Предобработка данных 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.random import seed
from numpy.random import randn
from scipy.stats import mannwhitneyu

In [2]:
data = pd.read_csv('/datasets/games.csv')
data

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/games.csv'

In [ ]:
data.info()

- Year_of_Release перевести в int
- User_Score перевести в float

In [ ]:
data.isna().sum()

- Пропуски в Name проверю есть ли там важная информация
- Пропуски в Year_of_Release заполню выбивающимся значением
- В Critic_Score, User_Score, Rating заменю на среднее по жанру

### Обработка названий столбцов <a id="columns"></a>

In [ ]:
data.columns = data.columns.str.lower()
data

- Привела названия столбцов к нижнему регистру.

### Удаление лишних строк  

In [ ]:
data['name'] = data['name'].fillna(value = 'Noname')

In [ ]:
df = data[data['name'] == 'Noname']
df

- В этих двух строках нет рейтинга, посмотрим есть ли еще игры 1993 года и какая информация по ним.

In [ ]:
df = data[data['year_of_release'] == 1993.0]
df

- Удаляю две строки с Noname, так как огромной ценности они не несут, а дополнить пропуски не вижу возможности.

In [ ]:
data = data.set_index('name')
data = data.drop('Noname')
data = data.reset_index()
data

In [ ]:
df = data[data['name'] == 'Noname']
df

- Избавилась от двух бесполезных строк. 

### Замена пропущенных значений

In [ ]:
data['rating'].unique()

- В столбце rating я заменю пропуски в играх до 2016 года на "E", то есть они будут помечены как "Для всех".
- Игры 2016 года будут помечены рейтингом "RP", так как они новые и могут ожидать оценки.

In [ ]:
data.loc[data['year_of_release'] == 2016, 'rating'] = data.loc[data['year_of_release'] == 2016, 'rating'].fillna('RP')
data.loc[data['year_of_release'] != 2016, 'rating'] = data.loc[data['year_of_release'] != 2016, 'rating'].fillna('E')
data.isna().sum()

- Избавилась от пропущенных значений в rating

In [ ]:
data['user_score'] = pd.to_numeric(data['user_score'], errors = 'coerce')

In [ ]:
data.isna().sum()

- Поменяла тип данных в столбце user_score на float64
- Заменила пропуски в critic_score и user_score на среднее по жанру
- Заменила пропуски в year_of_release на неправдоподобное значение чтобы не терять остальные важные параметры этих строк. Так я смогу отсеять при дальнейшем анализе эти значения, если понадобится.

### Замена типов данных 

In [ ]:
data.head()

In [ ]:
data.info()

- Все типы данных теперь в порядке

In [ ]:
data['year_of_release'].astype(pd.Int64Dtype())

### Создание дополнительного столбца  <a id="new_column"></a>

In [ ]:
data['total_sales'] = data['na_sales'] + data['eu_sales'] + data['jp_sales']
data

- Добавила столбец с общей выручкой

### Вывод <a id="total_1"></a>

Пропущенные значения в столбцах critic_score и user_score я заменила на среднее значение по жанру. Предполагаю, что пропущенные значения могли появиться из-за отсутствия данных за 1990-е года. 

Так же в столбце user_score мной были обнаружены значения "tbd", оно означает, что решение по этому критерию пока не принято. Так что и их я заменила на среднее по жанру, чтобы не терять такую большую часть данных.

Так же я удалила 2 строки, которые не содержали в себе никакой информации кроме года выпуска и продаж, я посчитала их ненужными. 

В столбцах user_score и year_of_release я сменила типы данных для облегчения дальнейшего анализа.

Добавила столбец total_sales с суммарными продажами по всем регионам.

## Исследовательский анализ данных 

### Анализ продаж по годам 

In [ ]:
data.head()

In [ ]:
data_years = pd.pivot_table(data, index='year_of_release', values = 'name', aggfunc = 'count')
data_years.columns = ['games_count']
data_years = data_years.query('year_of_release != 1000').plot(kind='bar', figsize = (8,10))
plt.show()

- На гистограмме я вижу неравномерное распледеление, смещенное вправо.
- Делаю вывод, что игры с 1980 года по 1990 год можно не использовать для анализа, так как это явный выброс, который можно отсеять.
- Так же с помощью метода query я убрала из сводной таблицы игры 1000-го года выпуска, так как эти строки для данного анализа непригодны.

### Анализ самых успешных платформ 

In [ ]:
data_platform = pd.pivot_table(data, index=['platform'], values = 'total_sales', aggfunc = 'sum').sort_values('total_sales', ascending=False)
data_platform_top = data_platform.head(18)
data_platform_top

In [ ]:
data = data.set_index('platform')
top_platforms_pivot = data[data.index.isin(data_platform_top.index)]
top_platforms_pivot

In [ ]:
top_platforms_ten = pd.pivot_table(top_platforms_pivot, index = ['platform', 'year_of_release'], values = 'total_sales', aggfunc = 'sum').query('year_of_release != 1000').reset_index()
top_platforms_ten.head()

In [ ]:
plt.figure(figsize=(20, 10))
sns.lineplot(x='year_of_release', y='total_sales', hue='platform', data=top_platforms_ten.query('year_of_release >= 1991'))
plt.xlabel('Year',fontsize=16)
plt.ylabel('Total sale',fontsize=16)
plt.title('Top 6 platforms')
plt.show()

- Я выделила топ 18 платформ, с самой высокой прибылью за все время.
- В среднем цикл жизни одной платформы занимает около 10 лет. Исключением из правил можно считать платформу DS, которая "прожила" больше 30 лет, и PC, она все еще работает с середины 1990-х.


Анализ основных игровых платформ показал:
- Три лидирующие по продажам платформы - PC, PS3, Wii
- Платформы PS4 и XOne начинают набирать рост, можно считать их потенциально прибыльными
- Из платформ, продолжающих работать в 2016 году на спад идут Wii, X360, PS3, 3DS и PC

In [ ]:
top_platforms_ten = top_platforms_ten.query('year_of_release >= 2005')
top_platforms_pivot = top_platforms_pivot.query('year_of_release >= 2005')

In [ ]:
plt.figure(figsize=(12, 9))
sns.boxplot(x='platform', y='total_sales', data = top_platforms_ten)
plt.show()

- Платформы PS2, PS3, PS4 можно считать самыми успешными на рынке компьютерных игр. Учитывая, что все эти платформы продукт одной линейк "Play station", вместе они составляют самое большое количество продаж.
- Платформы DS, Wii и X360 так же занимают лидирующие позиции по количеству продаж.

In [ ]:
top_platforms_pivot['user_score'] = top_platforms_pivot['user_score'] * 10
pd.set_option('chained_assignment', None)
top_platforms_pivot.head()

- Умножила показатели в столбце user_score на 10 для удобства сравнения.

### Анализ зависимости продаж от оценок пользователей и критиков

In [ ]:
top_platforms_pivot = top_platforms_pivot.reset_index()
ps2_all = top_platforms_pivot[top_platforms_pivot['platform'] == 'PS2']
ps2_all.head()

In [ ]:
fig, ax = plt.subplots(2, figsize=(10, 10))

ax[0].scatter(x = ps2_all['critic_score'], y = ps2_all['total_sales'])
ax[0].set_xlabel("critic_score")
ax[0].set_ylabel("total_sales")

ax[1].scatter(x = ps2_all['user_score'], y = ps2_all['total_sales'])
ax[1].set_xlabel("user_score")
ax[1].set_ylabel("total_sales")

plt.show()

In [ ]:
print('Корреляция между отзывами критиков и продажами:', ps2_all['critic_score'].corr(ps2_all['total_sales']))
print('Корреляция между отзывами пользователей и продажами:', ps2_all['user_score'].corr(ps2_all['total_sales']))

In [ ]:
wii_all = top_platforms_pivot[top_platforms_pivot['platform'] == 'Wii']

fig, ax = plt.subplots(2, figsize=(10, 10))

ax[0].scatter(x = wii_all['critic_score'], y = wii_all['total_sales'])
ax[0].set_xlabel("critic_score")
ax[0].set_ylabel("total_sales")

ax[1].scatter(x = wii_all['user_score'], y = wii_all['total_sales'])
ax[1].set_xlabel("user_score")
ax[1].set_ylabel("total_sales")

plt.show()

In [ ]:
print('Корреляция между отзывами критиков и продажами:', wii_all['critic_score'].corr(wii_all['total_sales']))
print('Корреляция между отзывами пользователей и продажами:', wii_all['user_score'].corr(wii_all['total_sales']))

In [ ]:
ds_all = top_platforms_pivot[top_platforms_pivot['platform'] == 'DS']

fig, ax = plt.subplots(2, figsize=(10, 10))

ax[0].scatter(x = ds_all['critic_score'], y = ds_all['total_sales'])
ax[0].set_xlabel("critic_score")
ax[0].set_ylabel("total_sales")

ax[1].scatter(x = ds_all['user_score'], y = ds_all['total_sales'])
ax[1].set_xlabel("user_score")
ax[1].set_ylabel("total_sales")

plt.show()

In [ ]:
print('Корреляция между отзывами критиков и продажами:', ds_all['critic_score'].corr(ds_all['total_sales']))
print('Корреляция между отзывами пользователей и продажами:', ds_all['user_score'].corr(ds_all['total_sales']))

In [ ]:
x360_all = top_platforms_pivot[top_platforms_pivot['platform'] == 'X360']
fig, ax = plt.subplots(2, figsize=(10, 10))

ax[0].scatter(x = x360_all['critic_score'], y = x360_all['total_sales'])
ax[0].set_xlabel("critic_score")
ax[0].set_ylabel("total_sales")

ax[1].scatter(x = x360_all['user_score'], y = x360_all['total_sales'])
ax[1].set_xlabel("user_score")
ax[1].set_ylabel("total_sales")

plt.show()

In [ ]:
print('Корреляция между отзывами критиков и продажами:', x360_all['critic_score'].corr(x360_all['total_sales']))
print('Корреляция между отзывами пользователей и продажами:', x360_all['user_score'].corr(x360_all['total_sales']))

Выделила для анализа азвисимости продаж от оценок несколько самых популярных платформ : PS2, Wii, DS и X360.

- Я наблюдаю общее повышение количества продаж на отметке 60 (critic_score - 60, user_score - 6)
- Примечательно, что после отметки в 80 баллов продажи снижаются и к 100 баллам почти сходят на нет
- Судя по коэффициенту корреляции отзывы критиков имеют влияние на продажи, в отличие от отзывов пользователей

In [ ]:
psp_all = top_platforms_pivot[top_platforms_pivot['platform'] == 'PSP']
fig, ax = plt.subplots(2, figsize=(10, 10))

ax[0].scatter(x = psp_all['critic_score'], y = psp_all['total_sales'])
ax[0].set_xlabel("critic_score")
ax[0].set_ylabel("total_sales")

ax[1].scatter(x = psp_all['user_score'], y = psp_all['total_sales'])
ax[1].set_xlabel("user_score")
ax[1].set_ylabel("total_sales")

plt.show()

In [ ]:
print('Корреляция между отзывами критиков и продажами:', psp_all['critic_score'].corr(psp_all['total_sales']))
print('Корреляция между отзывами пользователей и продажами:', psp_all['user_score'].corr(psp_all['total_sales']))

In [ ]:
xb_all = top_platforms_pivot[top_platforms_pivot['platform'] == 'XB']
fig, ax = plt.subplots(2, figsize=(10, 10))

ax[0].scatter(x = xb_all['critic_score'], y = xb_all['total_sales'])
ax[0].set_xlabel("critic_score")
ax[0].set_ylabel("total_sales")

ax[1].scatter(x = xb_all['user_score'], y = xb_all['total_sales'])
ax[1].set_xlabel("user_score")
ax[1].set_ylabel("total_sales")

plt.show()

In [ ]:
print('Корреляция между отзывами критиков и продажами:', xb_all['critic_score'].corr(xb_all['total_sales']))
print('Корреляция между отзывами пользователей и продажами:', xb_all['user_score'].corr(xb_all['total_sales']))

- У менее популярных платформ рост прибыли меняется так же, как и у популярных
- Однако коэффициент корреляции показывает что и отзывы критиков и отзывы пользователей влияют на повышение продаж

### Анализ зависимости продаж от жанров

In [ ]:
genres_pivot = pd.pivot_table(top_platforms_pivot, index = 'genre', values = 'total_sales', aggfunc = 'sum').sort_values(by='total_sales', ascending = False)
genres_pivot


In [ ]:
genres_pivot = genres_pivot.plot(kind='bar', figsize = (8,10))
plt.show()

- Самыми популярными жанрами являются Action и Sports
- Одними из самых неприбыльных жанров можно считать Puzzle, Adventure и Strategy

### Вывод 

Для анализа я использовала игры, выпущенные в период с 1991 года и до 2016. В некоторых этапах анализа данных я не использовала игры с пометкой 1000 в столбце с годом выпуска. 

Из всего представленного списка платформ я выделила 18 самых успешных и подходящих для анализа. В ходе анализа мной были выявлены основные критерии и показатели:
- В среднем цикл жизни одной платформы составляет около 10 лет, за исключением некоторых платформ, чья работа не прекращается и на протяжении 30 лет, но их не много.
- PC, PS3, Wii - лидирующие по общим продажам платформы.
- Потенциально прибыльными платформами можно считать платформы PS4 и XOne, так как их цикл жизни только начался и продажи начинают набирать обороты.
- Платформы Wii, X360, PS3, 3DS и PC продолжают работать в 2016 году, но их продажи уже начали идти на спад, предполагаю скорое завершение их жизненного цикла.
- Платформы линейки "Play station"(PS, PS2, PS3, PS4) можно считать самыми успешными из всех представленных, они занимают лидирующие позиции по количеству продаж. Так же в списке лидеров по продажам DS, Wii и X360.

Для анализа зависимости продаж от оценок я выбрала 4 популярные платформы и три менее популярные. Для облегчения сравнения двух показателей(оценки пользователей и критиков) я умножила на 10 оценки пользователей. Диаграммы показали повышение количества продаж на отметке в 60 баллов и их спад начиная с отметки 80 и дальше. Коэффициент корреляции у популярных платформ показал зависимость продаж от оценки критиков, в то время как у менее популярных платформ оценки критиков и пользователей имеют почти равное влияние на продажи. То есть, чем выше оценка, тем выше продажи.

Что касается жанров, лидирующие позиции по продажам занимают игры в жанре Action и Sports, а игры в жанре Puzzle, Adventure и Strategy занимают последние места в рейтинге.

## Портрет пользователя каждого региона 

### Северная америка (NA) 

In [ ]:
na_pivot = pd.pivot_table(top_platforms_pivot, index = 'platform', values = ['na_sales'], aggfunc = 'sum').sort_values(by='na_sales', ascending = False)
na_pivot.head()

In [ ]:
na_pivot.plot(kind='bar', figsize = (8,10))
plt.show()

In [ ]:
na_sum = na_pivot['na_sales'].sum()
print('Общие с продажи по Северной Америке составляют:', na_sum, 'млн. копий.')

In [ ]:
na_genre_pivot = pd.pivot_table(top_platforms_pivot, index = 'genre', values = ['na_sales'], aggfunc = 'sum').sort_values(by='na_sales', ascending = False)
na_genre_pivot.plot(kind='bar', figsize = (8,10))
plt.show()

In [ ]:
na_rating_pivot = pd.pivot_table(top_platforms_pivot, index = 'rating', values = ['na_sales'], aggfunc = 'sum').sort_values(by='na_sales', ascending = False)
na_rating_pivot.plot(kind='bar', figsize = (8,10))
plt.show()

In [ ]:
na_rating_pivot

- X360, Wii, PS3, DS и PS2 - самые продаваемые платформs
- Общие продажи по региону: 2578.2
- Action, Sports, Shooter, Misc и Role-Playing - самые популярные жанры
- Самые популярные игры с рейтингом E("Для всех"). Игры с рейтингом T("Подросткам") и M("Для взрослых") и E10+("От 10 лет и старше") так же на лидирующих позициях.

### Европа (EU) 

In [ ]:
eu_pivot = pd.pivot_table(top_platforms_pivot, index = 'platform', values = ['eu_sales'], aggfunc = 'sum').sort_values(by='eu_sales', ascending = False)
na_pivot.plot(kind='bar', figsize = (8,10))
plt.show()

In [ ]:
eu_sum = eu_pivot['eu_sales'].sum()
print('Общие с продажи по Европе составляют:', eu_sum, 'млн. копий.')

In [ ]:
eu_genre_pivot = pd.pivot_table(top_platforms_pivot, index = 'genre', values = ['eu_sales'], aggfunc = 'sum').sort_values(by='eu_sales', ascending = False)
eu_genre_pivot.plot(kind='bar', figsize = (8,10))
plt.show()

In [ ]:
eu_rating_pivot = pd.pivot_table(top_platforms_pivot, index = 'rating', values = ['eu_sales'], aggfunc = 'sum').sort_values(by='eu_sales', ascending = False)
eu_rating_pivot.plot(kind='bar', figsize = (8,10))
plt.show()

In [ ]:
eu_rating_pivot

- X360, Wii, PS3, DS и PS2 - самые популярные платформы
- Action, Sports, Shooter, Misc и Racing - самые популярные жанры 
- Общие продажи по региону: 1578.9
- Самые популярные игры с рейтингом E("Для всех"). Игры с рейтингом M("Для взрослых") и T("Подросткам") и E10+("От 10 лет и старше") так же на лидирующих позициях.

### Япония (JP) <a id="jp"></a>

In [ ]:
jp_pivot = pd.pivot_table(top_platforms_pivot, index = 'platform', values = ['jp_sales'], aggfunc = 'sum').sort_values(by='jp_sales', ascending = False)
jp_pivot.plot(kind='bar', figsize = (8,10))
plt.show()

In [ ]:
jp_sum = jp_pivot['jp_sales'].sum()
print('Общие с продажи по Японии составляют:', jp_sum, 'млн. копий.')

In [ ]:
jp_genre_pivot = pd.pivot_table(top_platforms_pivot, index = 'genre', values = ['jp_sales'], aggfunc = 'sum').sort_values(by='jp_sales', ascending = False)
jp_genre_pivot.plot(kind='bar', figsize = (8,10))
plt.show()

In [ ]:
jp_rating_pivot = pd.pivot_table(top_platforms_pivot, index = 'rating', values = ['jp_sales'], aggfunc = 'sum').sort_values(by='jp_sales', ascending = False)
jp_rating_pivot.plot(kind='bar', figsize = (8,10))
plt.show()

In [ ]:
jp_rating_pivot

- DS, 3DS, PS3, PSP и Wii - самые популярные платформы
- Общие продажи по региону: 580.6
- Role-Playing, Action, Misc, Sports и Platform - самые популярные жанры
- Самые популярные игры с рейтингом E("Для всех"). Игры с рейтингом T("Подросткам") и M("Для взрослых") и E10+("От 10 лет и старше") так же на лидирующих позициях.

### Вывод 

###### Северная Америка

- Самый крупный потребитель компьютерных игр
- Топ 5 самых популярных платформ: X360, PS2, Wii, PS3 и DS
- Топ 5 самых популярных жанров: Action, Sports, Shooter, Platform и Misc
- Игры с рейтингом E("Для всех") самые продаваемые, следующие по популярности - T("Подросткам"), M("Для взрослых") и E10+("От 10 лет и старше")

###### Европа

- Топ 5 самых популярных платформ: X360, PS2, Wii, PS3 и DS
- Топ 5 самых популярных жанров: Action, Sports, Shooter, Racing и Misc
- Игры с рейтингом E("Для всех") самые продаваемые, следующие по популярности - M("Для взрослых"), T("Подросткам") и E10+("От 10 лет и старше")

###### Япония

- Топ 5 самых популярных платформ: DS, PS, PS2, SNES и 3DS
- Топ 5 самых популярных жанров: Role-Playing, Action, Sports, Platform и Misc
- Игры с рейтингом E("Для всех") самые продаваемые, следующие по популярности - T("Подросткам"), M("Для взрослых") и E10+("От 10 лет и старше")

Можно сказать, что платформы X360, PS2, Wii, PS3 и DS занимают лидирующие позиции в мире. При этом, если в Европе и Северной Америке на первых местах расположилась X360, а DS на последнем, то в Японии DS занимает первое место, а X360 вообще не входит в топ пять. Это указывает на особенность выбора японского потребителя. 

Так же можно заметить, что популярные жанры игр в Северной Америке и Европе почти схожи. Но японский пользователь отдает предпочтение играм в жанре Role-Playing, что так же подтверждает особенность пользователей данного региона. 

Во всех трех регионах рейтинг E("Для всех") самый популярный, но стоит заметить, что в Евроме на второе место выходят игры с рейтингом M("Для взрослых"). 

## Проверка гипотез 

### Гипотеза №1 
###### Нулевая гипотеза: 
    Средние пользовательские рейтинги платформ Xbox One и PC одинаковые.

###### Альтернативная гипотеза: 
    Средние пользовательские рейтинги платформ Xbox One и PC разные.

In [ ]:
xbox_one = top_platforms_pivot[top_platforms_pivot['platform'] == 'XOne']
xbox_one['user_score'].mean()

In [ ]:
pc_ = top_platforms_pivot[top_platforms_pivot['platform'] == 'PC']
pc_['user_score'].mean()

In [ ]:
seed(1)

data1 = xbox_one['user_score']
data2 = pc_['user_score']

stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))

alpha = 0.05

if p > alpha:
    print('Принимаем нулевую гипотезу')
else:
    print('Отвергаем нулевую гипотезу')

### Гипотеза №2 
###### Нулевая гипотеза: 
    Средние пользовательские рейтинги жанров Action и Sports одинаковые.

###### Альтернативная гипотеза: 
    Средние пользовательские рейтинги жанров Action и Sports разные.

In [ ]:
action = top_platforms_pivot[top_platforms_pivot['genre'] == 'Action']
action['user_score'].mean()

In [ ]:
sports = top_platforms_pivot[top_platforms_pivot['genre'] == 'Sports']
sports['user_score'].mean()

In [ ]:
seed(1)

data1 = action['user_score']
data2 = sports['user_score']

stat, p = mannwhitneyu(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))

alpha = 0.05

if p > alpha:
    print('Принимаем нулевую гипотезу')
else:
    print('Отвергаем нулевую гипотезу')

- Нулевую и альтернативную гипотезы я формулировала по принципу "положительно-отрицательно", то есть нулевая гипотеза у меня "положительная", а альтернативная - "отрицательная"
- Для проверки гипотез я применила U-тест Манна-Уитни. Так как распределение было не нормальным, я не смогла применить Т-критерий Стьюдента. 

## Общий вывод 

Основной задачей проекта было выявление закономерностей, определяющих успешность компьютерной игры. 

Исходя из проведенного анализа я выделила основные закономерности:
1. Жанры, обреченные на успех: Action, Sports, Shooter, Role-Playing
2. Перспективные платформы: PS4 и XOne
3. Самые прибыльные игры с рейтингом: E, T, M, E10+
4. Самые популярные платформы: X360, PS2, Wii, PS3, DS 

Стоить упомянуть об особенностях рынка компьютерных игр в разных регионах:
- Пользователи из Японии предпочитают игры жанра Role-Playing и платформу DS
- У пользователей из Европы игры с рейтингом M стоят на втором месте, после всемирно-популярного E. 
- Североамериканский рынок компьютерных игр самый крупный из трех представленных, больше, чем японский и европейский вместе взятые. 

Коэффициент корреляции показал, что на популярных, крупных платформах, на продажи копий влияют только оценки критиков. В менее популярных и крупных платформах оценки критиков и пользователей одинаково вляют на продажи.

Из всего вышесказанного можно примерно составить портрет самой популярной и прибыльной игры в 2017 году:
- Платформа: PS4
- Жанр: Action
- Рейтинг ESRB: E